In [1]:
import spacy
import numpy as np
import pandas as pd
from spacy import displacy


nlp = spacy.load("en_core_web_sm")
print("Pipeline:", nlp.pipe_names)

/home/acro0/NLP/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


# Token-based matching

spaCy features a rule-matching engine, the Matcher, that operates over tokens, similar to regular expressions. The rules can refer to token annotations (e.g. the token text or tag_, and flags like IS_PUNCT).

In [10]:
from spacy.matcher import Matcher
from spacy.tokens import Span

nlp_e = spacy.blank("en")
matcher = Matcher(nlp_e.vocab)
matcher.add("PERSON", [[{"lower": "barack"}, {"lower": "obama"}]])
doc = nlp_e("Barack Obama was the 44th president of the United States")

# 1. Return (match_id, start, end) tuples
matches = matcher(doc)
for match_id, start, end in matches:
    # Create the matched span and assign the match_id as a label
    span = Span(doc, start, end, label=match_id)
    print(span.text, span.label_)

# 2. Return Span objects directly
matches = matcher(doc, as_spans=True)
for span in matches:
    print(span.text, span.label_)

Barack Obama PERSON
Barack Obama PERSON


In [9]:
import re

doc = nlp("""The United States of America (USA) are commonly 
            known as the United States (U.S. or US) or America.""")

expression = r"[Uu](nited|\\.?) ?[Ss](tates|\\.?)"

for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    # This is a Span object or None if match doesn't map to valid token sequence
    if span is not None:
        print("Found match:", span.text)


Found match: United States
Found match: United States


# Efficient phrase matching

If you need to match large terminology lists, you can also use the PhraseMatcher and create Doc objects instead of token patterns, which is much more efficient overall. The Doc patterns can contain single or multiple tokens.

In [11]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)
terms = ["Barack Obama", "Angela Merkel", "Washington, D.C."]
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)

doc = nlp("German Chancellor Angela Merkel and US President Barack Obama "
          "converse in the Oval Office inside the White House in Washington, D.C.")
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

Angela Merkel
Barack Obama
Washington, D.C.


In [12]:
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

nlp_e = English()
matcher = PhraseMatcher(nlp_e.vocab, attr="SHAPE")
matcher.add("IP", [nlp_e("127.0.0.1"), nlp_e("127.127.0.0")])

doc = nlp_e("Often the router will have an IP address such as 192.168.1.1 or 192.168.2.1.")
for match_id, start, end in matcher(doc):
    print("Matched based on token shape:", doc[start:end])


Matched based on token shape: 192.168.1.1
Matched based on token shape: 192.168.2.1


# Dependency Matcher

The DependencyMatcher lets you match patterns within the dependency parse using Semgrex operators. It requires a model containing a parser such as the DependencyParser. 

In [13]:
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)
pattern = [
    {
        "RIGHT_ID": "anchor_founded",
        "RIGHT_ATTRS": {"ORTH": "founded"}
    },
    {
        "LEFT_ID": "anchor_founded",
        "REL_OP": ">",
        "RIGHT_ID": "founded_subject",
        "RIGHT_ATTRS": {"DEP": "nsubj"},
    },
    {
        "LEFT_ID": "anchor_founded",
        "REL_OP": ">",
        "RIGHT_ID": "founded_object",
        "RIGHT_ATTRS": {"DEP": "dobj"},
    },
    {
        "LEFT_ID": "founded_object",
        "REL_OP": ">",
        "RIGHT_ID": "founded_object_modifier",
        "RIGHT_ATTRS": {"DEP": {"IN": ["amod", "compound"]}},
    }
]

matcher.add("FOUNDED", [pattern])
doc = nlp("Lee, an experienced CEO, has founded two AI startups.")
matches = matcher(doc)

print(matches) # [(4851363122962674176, [6, 0, 10, 9])]
# Each token_id corresponds to one pattern dict
match_id, token_ids = matches[0]
for i in range(len(token_ids)):
    print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text)


[(4851363122962674176, [7, 0, 10, 9])]
anchor_founded: founded
founded_subject: Lee
founded_object: startups
founded_object_modifier: AI


# Rule-based entity recognition

The EntityRuler is a component that lets you add named entities based on pattern dictionaries, which makes it easy to combine rule-based and statistical named entity recognition for even more powerful pipelines.

In [14]:


nlp_e = English()
ruler = nlp_e.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "Apple", "id": "apple"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}], "id": "san-francisco"},
            {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "fran"}], "id": "san-francisco"}]
ruler.add_patterns(patterns)

data1 = []
doc1 = nlp_e("Apple is opening its first big office in San Francisco.")
for ent in doc1.ents:
    data1.append([ent.text, ent.label_, ent.ent_id_])

data2 = []
doc2 = nlp_e("Apple is opening its first big office in San Fran.")
for ent in doc2.ents:
    data2.append([ent.text, ent.label_, ent.ent_id_])

df = pd.DataFrame({"Doc1":data1,"Doc2":data2})
df


,Doc1,Doc2
0,"[Apple, ORG, apple]","[Apple, ORG, apple]"
1,"[San Francisco, GPE, san-francisco]","[San Fran, GPE, san-francisco]"


# Rule-based span matching

The SpanRuler is a generalized version of the entity ruler that lets you add spans to doc.spans or doc.ents based on pattern dictionaries, which makes it easy to combine rule-based and statistical pipeline components.

In [21]:
config = {"spans_key": None, "annotate_ents": True, "overwrite": False}
ruler = nlp.add_pipe("span_ruler", config=config)
patterns = [{"label": "ORG", "pattern": "MyCorp Inc."}]
ruler.add_patterns(patterns)

doc = nlp("MyCorp Inc. is a company in the U.S.")
print([(ent.text, ent.label_) for ent in doc.ents])


[('MyCorp Inc.', 'ORG'), ('U.S.', 'GPE')]


In [25]:
#